In [2]:
import pandas as pd
import numpy as np

In [3]:
# https://www.kaggle.com/competitions/digit-recognizer/

In [4]:
train_df = pd.read_csv("./digit-recognizer/train.csv")

In [5]:
test_df = pd.read_csv("./digit-recognizer/test.csv")

In [6]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

In [7]:
train_df, val_df = train_test_split(train_df)
train_X = torch.as_tensor(train_df.values)[:,1:]
train_y = torch.as_tensor(train_df.values)[:,0]
val_X = torch.as_tensor(val_df.values)[:,1:]
val_y = torch.as_tensor(val_df.values)[:,0]
test_X = torch.as_tensor(test_df.values)
print(train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape)

torch.Size([31500, 784]) torch.Size([31500]) torch.Size([10500, 784]) torch.Size([10500]) torch.Size([28000, 784])


In [8]:
train_dl = DataLoader(TensorDataset(train_X, train_y), batch_size=128, shuffle=True, num_workers=4)
val_dl = DataLoader(TensorDataset(val_X, val_y), batch_size=128, shuffle=False, num_workers=4)
test_dl = DataLoader(TensorDataset(test_X), batch_size=128, shuffle=False, num_workers=4)

In [9]:
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1, stride=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1, stride=1)
        self.conv3 = nn.Conv2d(64,128, 3, padding=1, stride=1)
        self.flatten = nn.Flatten(1,-1)
        self.fc1 = nn.Linear(128*3*3, 10)
        nn.init.kaiming_normal_(self.conv1.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.conv2.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.conv3.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.fc1.weight, nonlinearity='relu')


    def forward(self, x):
        x = x.view(-1, 1, 28, 28)
        x = x/255
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.fc1(x)

        return x


net = Net()


In [10]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-3, betas=(0.9, 0.98))


In [11]:
train_loader = train_dl
val_loader = val_dl

In [12]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-3, betas=(0.9, 0.95))
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # print(type(inputs), type(labels))
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        # print(type(outputs))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
        # break
    correct = 0
    total = 0

    with torch.no_grad():
        for data in val_loader:
            images, labels = data
            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the validation images: {100 * correct / total} %')

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7faa58e0a310>
Traceback (most recent call last):
  File "/Users/levi/opt/anaconda3/envs/DeepLearning/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1465, in __del__
    def __del__(self):
  File "/Users/levi/opt/anaconda3/envs/DeepLearning/lib/python3.8/site-packages/torch/utils/data/_utils/signal_handling.py", line 66, in handler
    _error_if_any_worker_fails()
RuntimeError: DataLoader worker (pid 26941) is killed by signal: Interrupt: 2. 


KeyboardInterrupt: 

In [ ]:
test_dl = DataLoader(test_X, batch_size=128, shuffle=False, num_workers=4)

In [ ]:
with torch.no_grad():
    predicted_set = []
    for data in test_dl:
        images = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        predicted_set.append(predicted)

In [ ]:
predicted_set

In [ ]:
a = []
for i in predicted_set:
    for j in i:
      a.append(j.item())

In [ ]:
a

In [ ]:
len(a)

In [ ]:
b = range(1,28000+1)

In [ ]:
b = np.arange(1,28000+1)

In [ ]:
data = np.array([b,a]).transpose()

In [ ]:
res = pd.DataFrame(data, columns=['ImageId', 'Label'])

In [ ]:
res.to_csv('prediction.csv',index=False)